# Experiment 3: Time-Aware Baseline Model
Using only hour of day and station ID.

In [2]:
!pip install capymoa

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 30.7 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=bb49c558e9a91aeaeb789c6b53d3ee421eda5795cffda58a4a12b7fdbe2185da
  Stored in directory: /root/.cache/pip/wheels/40/b3/0f/a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
from capymoa.regressor import FIMTDD
from capymoa.stream import  Schema

In [9]:
# Load dataset
df = pd.read_csv('/content/loader_f30avg_new.csv')
df['datetime'] = pd.to_datetime(df['timestamp'])
df['hour'] = df['datetime'].dt.hour
df['station'] = df['stop'].astype(str)

In [11]:
# Prepare features and target
features = df[['hour', 'station']]
target = df['f30avg']

encoder = OneHotEncoder(sparse_output=False)
X_encoded = encoder.fit_transform(features)

X_array = X_encoded.astype(np.float32)
y_array = target.to_numpy().astype(np.float32)

stream = DataStream(X_array, y_array)

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

In [ ]:
# Initialize model and delayed evaluation
model = FIMTDD()
true_values = []
predictions = []
buffer_X = []
buffer_y = []

for x_i, y_i in stream:
    if len(buffer_X) >= 6:
        pred = model.predict([buffer_X.pop(0)])[0]
        true = buffer_y.pop(0)
        predictions.append(pred)
        true_values.append(true)

    model.partial_fit([x_i], [y_i])
    buffer_X.append(x_i)
    buffer_y.append(y_i)

In [ ]:
# Results
mae = mean_absolute_error(true_values, predictions)
rmse = sqrt(mean_squared_error(true_values, predictions))

print(f"MAE: {mae:.3f}")
print(f"RMSE: {rmse:.3f}")